In [1]:
!pip install transformers
!pip install datasets
!pip install tokenizer
!pip install seqeval

In [2]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification #This line imports the AutoModelForTokenClassification class from the Hugging Face Transformers library. This class is designed to load various pre-trained models for token classification tasks.

conll2003 dataset loading : https://huggingface.co/datasets/conll2003

In [3]:
conll2003 = datasets.load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
conll2003["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [6]:
conll2003["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [7]:
conll2003["train"].features["ner_tags"]       # ner - name entity recognisation tag to identify the names places etc., in the given text

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
conll2003["train"].description

'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

In [9]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


In [11]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [12]:
example_text = conll2003['train'][0]

In [13]:
example_text

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [14]:
conll2003["train"].features["ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [15]:
example_text["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [16]:
tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)       # converting into tokenized numbers

In [17]:
tokenized_input

{'input_ids': [101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
tokenized_input['input_ids']  # collecting only input ids from the tokenized input

[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]

In [19]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])      # and again converting back to token

In [20]:
tokens    # at the starting will find 'CLS' means classificcation and at the end will find 'SEP'  means seperator

['[CLS]',
 'eu',
 'rejects',
 'german',
 'call',
 'to',
 'boycott',
 'british',
 'lamb',
 '.',
 '[SEP]']

In [21]:
word_ids = tokenized_input.word_ids()     # collecting the word_ids

In [22]:
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [23]:
example_text["ner_tags"]

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [24]:
for i, label in enumerate(example_text["ner_tags"]):        # getting the index values for the ner tags
  print(i,label)

0 3
1 0
2 7
3 0
4 0
5 0
6 7
7 0
8 0


In [25]:
tokenized_input.word_ids(0)

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]

In [26]:
def tokenize_and_align_labels(examples, label_all_tokens=True):

    #tokeinze ids
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []


    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.

        previous_word_idx = None
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)

            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [27]:
conll2003["train"][4:5]

{'id': ['4'],
 'tokens': [['Germany',
   "'s",
   'representative',
   'to',
   'the',
   'European',
   'Union',
   "'s",
   'veterinary',
   'committee',
   'Werner',
   'Zwingmann',
   'said',
   'on',
   'Wednesday',
   'consumers',
   'should',
   'buy',
   'sheepmeat',
   'from',
   'countries',
   'other',
   'than',
   'Britain',
   'until',
   'the',
   'scientific',
   'advice',
   'was',
   'clearer',
   '.']],
 'pos_tags': [[22,
   27,
   21,
   35,
   12,
   22,
   22,
   27,
   16,
   21,
   22,
   22,
   38,
   15,
   22,
   24,
   20,
   37,
   21,
   15,
   24,
   16,
   15,
   22,
   15,
   12,
   16,
   21,
   38,
   17,
   7]],
 'chunk_tags': [[11,
   11,
   12,
   13,
   11,
   12,
   12,
   11,
   12,
   12,
   12,
   12,
   21,
   13,
   11,
   12,
   21,
   22,
   11,
   13,
   11,
   1,
   13,
   11,
   17,
   11,
   12,
   12,
   21,
   1,
   0]],
 'ner_tags': [[5,
   0,
   0,
   0,
   0,
   3,
   4,
   0,
   0,
   0,
   1,
   2,
   0,
   0,
   0,
   0,
   0,


In [28]:
tokenize_and_align_labels(conll2003["train"][4:5])

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [29]:
q = tokenize_and_align_labels(conll2003["train"][4:5])

In [30]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
germany_________________________________ 5
'_______________________________________ 0
s_______________________________________ 0
representative__________________________ 0
to______________________________________ 0
the_____________________________________ 0
european________________________________ 3
union___________________________________ 4
'_______________________________________ 0
s_______________________________________ 0
veterinary______________________________ 0
committee_______________________________ 0
werner__________________________________ 1
z_______________________________________ 2
##wing__________________________________ 2
##mann__________________________________ 2
said____________________________________ 0
on______________________________________ 0
wednesday_______________________________ 0
consumers_______________________________ 0
should__________________________________ 0
buy_____________________________________ 0
sheep___

In [31]:
## Applying on entire data
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)

In [32]:
tokenized_datasets    # got input_ids and labels

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [33]:
tokenized_datasets["train"][0]      # got input_ids and labels

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [101,
  7327,
  19164,
  2446,
  2655,
  2000,
  17757,
  2329,
  12559,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]}

In [34]:
# This line creates an instance of the AutoModelForTokenClassification class and loads the pre-trained BERT model specified by the "bert-base-uncased"
# Additionally, it specifies that the token classification task has 9 labels (num_labels=9).
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased",num_labels=9)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# imports two classes, TrainingArguments and Trainer, from the Hugging Face Transformers library. These classes are used for training transformer-based models.
from transformers import TrainingArguments, Trainer

In [36]:
!pip install accelerate

In [37]:
!pip install transformers[torch]

In [38]:
args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=1,
weight_decay=0.01
)

In [39]:
# setting up a Trainer for training a transformer-based model using Hugging Face's Transformers library
Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

NameError: name 'data_collator' is not defined

In [40]:
# used for sequence labeling tasks such as named entity recognition (NER).
metric=datasets.load_metric("seqeval")

<ipython-input-40-76447e88b22e>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [41]:
example=conll2003['train'][0]

In [42]:
example

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [43]:
label_list = conll2003["train"].features["ner_tags"].feature.names

label_list  # labels list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [44]:
for i in example["ner_tags"]:   # corresponding encoded value for the labels
  print(i)

3
0
7
0
0
0
7
0
0


In [45]:
labels = []
for i in example["ner_tags"]:   # labels for the corresponding ner tags
    labels.append(label_list[i])

labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [46]:
# Here, predicted_labels and reference_labels are lists of labels for sequences. The compute method compares these two lists and calculates the metrics.
metric.compute(predictions=[labels],references=[labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [47]:
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }

In [48]:
data_collator=DataCollatorForTokenClassification(tokenizer)

In [49]:
trainer=Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [50]:
# fine tuning model
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.223600,0.069527,0.908185,0.927285,0.917635,0.981127


TrainOutput(global_step=878, training_loss=0.16398200032922836, metrics={'train_runtime': 157.1517, 'train_samples_per_second': 89.347, 'train_steps_per_second': 5.587, 'total_flos': 342221911376202.0, 'train_loss': 0.16398200032922836, 'epoch': 1.0})

In [51]:
model.save_pretrained("ner_model")

In [52]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [53]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [57]:
# id2label = {
#     str(i): label for i,label in enumerate(label_list)
# }

#or

id2label = {}
for i, label in enumerate(label_list):
    id2label[i] = label


In [58]:
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [60]:
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [61]:
label2id

{'O': '0',
 'B-PER': '1',
 'I-PER': '2',
 'B-ORG': '3',
 'I-ORG': '4',
 'B-LOC': '5',
 'I-LOC': '6',
 'B-MISC': '7',
 'I-MISC': '8'}

In [62]:
import json
config=json.load(open("/content/ner_model/config.json"))

In [63]:
config

{'_name_or_path': 'bert-base-uncased',
 'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'LABEL_0',
  '1': 'LABEL_1',
  '2': 'LABEL_2',
  '3': 'LABEL_3',
  '4': 'LABEL_4',
  '5': 'LABEL_5',
  '6': 'LABEL_6',
  '7': 'LABEL_7',
  '8': 'LABEL_8'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'LABEL_0': 0,
  'LABEL_1': 1,
  'LABEL_2': 2,
  'LABEL_3': 3,
  'LABEL_4': 4,
  'LABEL_5': 5,
  'LABEL_6': 6,
  'LABEL_7': 7,
  'LABEL_8': 8},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.35.2',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [66]:
config["id2label"] = id2label   # assigning the id2label lablelists to config

In [67]:
config["label2id"] = label2id    # assigning the label2id lablelists to config

In [68]:
json.dump(config,open("/content/ner_model/config.json","w"))  # writing the updated configs to same config.json file

In [69]:
# AutoModelForTokenClassification: This is a class from the transformers library for token classification models.
#  It automatically selects the appropriate model class based on the specified architecture.

# from_pretrained("ner_model"): This method loads a pre-trained model from the "ner_model" directory.
# The "ner_model" directory is assumed to contain the necessary model weights and configuration.

model_fine_tuned=AutoModelForTokenClassification.from_pretrained("ner_model")

In [70]:
model_fine_tuned

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

# transformer pipeline

In [72]:
from transformers import pipeline   # importing the pipeline from transformer

In [74]:
nlp_pipeline=pipeline("ner",model=model_fine_tuned,tokenizer=tokenizer) # paasing fine tuned model and tokenizer to transformer pipeline

In [75]:
nlp_pipeline

In [76]:
example="sudhanshu kumar is a foundar of iNeuron"

In [77]:
nlp_pipeline(example)

[{'entity': 'B-PER',
  'score': 0.99201936,
  'index': 1,
  'word': 'sud',
  'start': 0,
  'end': 3},
 {'entity': 'B-PER',
  'score': 0.99062854,
  'index': 2,
  'word': '##han',
  'start': 3,
  'end': 6},
 {'entity': 'B-PER',
  'score': 0.9921003,
  'index': 3,
  'word': '##shu',
  'start': 6,
  'end': 9},
 {'entity': 'I-PER',
  'score': 0.99556607,
  'index': 4,
  'word': 'kumar',
  'start': 10,
  'end': 15},
 {'entity': 'B-ORG',
  'score': 0.9693006,
  'index': 10,
  'word': 'in',
  'start': 32,
  'end': 34},
 {'entity': 'B-ORG',
  'score': 0.96259046,
  'index': 11,
  'word': '##eur',
  'start': 34,
  'end': 37},
 {'entity': 'B-ORG',
  'score': 0.97199124,
  'index': 12,
  'word': '##on',
  'start': 37,
  'end': 39}]

In [78]:
example="sunny is a founder of microsoft"

In [79]:
nlp_pipeline(example)

[{'entity': 'B-PER',
  'score': 0.98261917,
  'index': 1,
  'word': 'sunny',
  'start': 0,
  'end': 5},
 {'entity': 'B-ORG',
  'score': 0.7523059,
  'index': 6,
  'word': 'microsoft',
  'start': 22,
  'end': 31}]

In [80]:
example="apple launch mobile while eating apple which taste like orange"


In [81]:
nlp_pipeline(example)

[{'entity': 'B-ORG',
  'score': 0.92133886,
  'index': 1,
  'word': 'apple',
  'start': 0,
  'end': 5},
 {'entity': 'B-ORG',
  'score': 0.45179307,
  'index': 6,
  'word': 'apple',
  'start': 33,
  'end': 38}]

In [82]:
example="vikas is working ai engineer in google"

In [83]:
nlp_pipeline(example)

[{'entity': 'B-PER',
  'score': 0.9922707,
  'index': 1,
  'word': 'vi',
  'start': 0,
  'end': 2},
 {'entity': 'B-PER',
  'score': 0.9906538,
  'index': 2,
  'word': '##kas',
  'start': 2,
  'end': 5},
 {'entity': 'B-ORG',
  'score': 0.9267004,
  'index': 8,
  'word': 'google',
  'start': 32,
  'end': 38}]

In [84]:
example="apple founder loves eating apple"


In [85]:
nlp_pipeline(example)

[{'entity': 'B-ORG',
  'score': 0.9399445,
  'index': 1,
  'word': 'apple',
  'start': 0,
  'end': 5}]

In [86]:
example="Microsoft Windows created their software by idea that came from the window of the house"

In [87]:
nlp_pipeline(example)

[{'entity': 'B-ORG',
  'score': 0.9258155,
  'index': 1,
  'word': 'microsoft',
  'start': 0,
  'end': 9},
 {'entity': 'I-ORG',
  'score': 0.8877325,
  'index': 2,
  'word': 'windows',
  'start': 10,
  'end': 17}]

In [94]:
example="She will lead the lead team in the project."

In [95]:
nlp_pipeline(example)

[]

In [96]:
example="The plane was flying over the desert and the dessert was served."

In [97]:
nlp_pipeline(example)

[]

In [98]:
example = "The company Apple grows apples in Washington."

In [99]:
nlp_pipeline(example)

[{'entity': 'B-ORG',
  'score': 0.93989754,
  'index': 3,
  'word': 'apple',
  'start': 12,
  'end': 17},
 {'entity': 'B-LOC',
  'score': 0.9920677,
  'index': 7,
  'word': 'washington',
  'start': 34,
  'end': 44}]

In [108]:
example = "The soldier decided to desert his post in the desert."

In [109]:
nlp_pipeline(example)

[]